# Coherence Simulator Pack — Notebook

This notebook bundles four runnable demos:

1. **Semi-implicit diffusion (spectral)** — stable gradient flow of a phase field.
2. **Symplectic Sine–Gordon** — Hamiltonian, energy-bounded evolution.
3. **RG coarse-graining probe** — block-averaging Π_ℓ and stats across scales.
4. **Gauged RFT gradient flow** — phase–gauge coupling with curvature energy.

Run cells top-to-bottom. Plots appear inline and CSVs are saved under `/mnt/data/`.


In [ ]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt, os
os.makedirs('/mnt/data', exist_ok=True)

def wrap_phase(theta):
    import numpy as np
    return np.arctan2(np.sin(theta), np.cos(theta))

def laplacian_periodic(u, dx):
    import numpy as np
    return (np.roll(u,1,0)+np.roll(u,-1,0)+np.roll(u,1,1)+np.roll(u,-1,1)-4*u)/(dx*dx)

def grad_periodic(u, dx):
    import numpy as np
    ux = (np.roll(u,-1,1)-np.roll(u,1,1))/(2*dx)
    uy = (np.roll(u,-1,0)-np.roll(u,1,0))/(2*dx)
    return ux, uy

def div_periodic(vx, vy, dx):
    import numpy as np
    return (np.roll(vx,-1,1)-np.roll(vx,1,1))/(2*dx) + (np.roll(vy,-1,0)-np.roll(vy,1,0))/(2*dx)

def curl2d_periodic(ax, ay, dx):
    import numpy as np
    dAy_dx = (np.roll(ay,-1,1)-np.roll(ay,1,1))/(2*dx)
    dAx_dy = (np.roll(ax,-1,0)-np.roll(ax,1,0))/(2*dx)
    return dAy_dx - dAx_dy

def curlT2d_periodic(F, dx):
    import numpy as np
    dF_dx = (np.roll(F,-1,1)-np.roll(F,1,1))/(2*dx)
    dF_dy = (np.roll(F,-1,0)-np.roll(F,1,0))/(2*dx)
    return dF_dy, -dF_dx

def coherence(theta):
    import numpy as np
    z = np.exp(1j*theta)
    return float(np.abs(z.mean())**2)

def grad_sq_periodic(theta, dx):
    tx, ty = grad_periodic(theta, dx)
    return tx**2 + ty**2

def energy_ungauged(theta, kappa1, dx, V=None):
    g2 = grad_sq_periodic(theta, dx)
    E = 0.5*kappa1*g2.mean()
    if V is not None: E += V(theta).mean()
    return float(E)

def vortex_count(theta):
    import numpy as np
    th = theta
    d1 = wrap_phase(np.roll(th,-1,1)-th)
    d2 = wrap_phase(np.roll(th,-1,0)-np.roll(th,-1,1))
    d3 = wrap_phase(np.roll(th,1,1)-np.roll(th,-1,0))
    d4 = wrap_phase(th-np.roll(th,1,1))
    winding = wrap_phase(d1+d2+d3+d4)/(2*np.pi)
    return int(np.round(winding.sum()))

def init_vortex_pair(nx, ny, L, sep_frac=0.25):
    import numpy as np
    x = np.linspace(-L/2, L/2, nx, endpoint=False)
    y = np.linspace(-L/2, L/2, ny, endpoint=False)
    X, Y = np.meshgrid(x, y, indexing='ij')
    a = L*sep_frac
    centers = [(-a,0.0,+1), (+a,0.0,-1)]
    th = np.zeros((nx,ny),dtype=float)
    for (xc,yc,q) in centers:
        th += q*np.arctan2(Y-yc, X-xc)
    return wrap_phase(th)


In [ ]:

# 1) Semi-implicit diffusion (spectral)
nx=128; L=2*np.pi; kappa1=1.0; dt=1e-3; steps=12000; sample_every=200
dx=L/nx
kx = 2*np.pi*np.fft.fftfreq(nx, d=dx)
KX, KY = np.meshgrid(kx, kx, indexing='ij'); K2=KX**2+KY**2
theta = init_vortex_pair(nx,nx,L,sep_frac=0.25)
times=[]; Cs=[]; Es=[]
for step in range(steps+1):
    if step % sample_every==0:
        times.append(step*dt); Cs.append(coherence(theta)); Es.append(energy_ungauged(theta,kappa1,dx))
    th_hat = np.fft.fftn(theta)
    theta = np.fft.ifftn(th_hat/(1.0 + dt*kappa1*K2)).real
    theta = wrap_phase(theta)
import pandas as pd, matplotlib.pyplot as plt
df = pd.DataFrame({'t':times,'C':Cs,'E':Es}); df.to_csv('/mnt/data/nb_diffusion_timeseries.csv', index=False)
plt.figure(figsize=(7,5)); plt.plot(df.t, df.C); plt.xlabel('time'); plt.ylabel('C(t)'); plt.title('Semi-implicit diffusion'); plt.tight_layout(); plt.show()


In [ ]:

# 2) Symplectic Sine–Gordon
nx=128; L=2*np.pi; c=1.0; m2=1.0; dt=2e-3; steps=8000; sample_every=50
x=np.linspace(-L/2,L/2,nx,endpoint=False); X,Y=np.meshgrid(x,x,indexing='ij')
theta = wrap_phase(0.5*np.exp(-((X**2+Y**2)/(0.2**2)))); p=np.zeros_like(theta)
dx=L/nx
times=[]; Cs=[]; Hs=[]
def energy_sg(th,p):
    g2 = grad_sq_periodic(th, dx)
    V = m2*(1-np.cos(th))
    return 0.5*(p**2).mean() + 0.5*c*c*g2.mean() + V.mean()
for step in range(steps+1):
    if step % sample_every==0:
        times.append(step*dt); Cs.append(coherence(theta)); Hs.append(energy_sg(theta,p))
    lap = laplacian_periodic(theta, dx)
    p = p + 0.5*dt*( c*c*lap - m2*np.sin(theta) )
    theta = wrap_phase(theta + dt*p)
    lap2 = laplacian_periodic(theta, dx)
    p = p + 0.5*dt*( c*c*lap2 - m2*np.sin(theta) )
df = pd.DataFrame({'t':times,'C':Cs,'H':Hs}); df.to_csv('/mnt/data/nb_sinegordon_timeseries.csv', index=False)
plt.figure(figsize=(7,5)); plt.plot(df.t, df.H); plt.xlabel('time'); plt.ylabel('H(t)'); plt.title('Sine–Gordon (symplectic)'); plt.tight_layout(); plt.show()


In [ ]:

# 3) RG coarse-graining probe
def block_average(u): return 0.25*(u[0::2,0::2]+u[1::2,0::2]+u[0::2,1::2]+u[1::2,1::2])
nx=128; L=2*np.pi; theta0 = init_vortex_pair(nx,nx,L,sep_frac=0.22)
kappa1=1.0; records=[]; th=theta0.copy(); cur=nx; dx=L/nx; level=0; levels=int(np.log2(nx))-4
while cur>=16 and level<=levels:
    C=coherence(th); E=energy_ungauged(th,kappa1,dx); vort=vortex_count(th)
    records.append({'level':level,'nx':cur,'dx':dx,'C':C,'E':E,'vortex_net':vort})
    th = wrap_phase(block_average(th)); cur = th.shape[0]; dx=L/cur; level+=1
df = pd.DataFrame(records); df.to_csv('/mnt/data/nb_rg_summary.csv', index=False)
plt.figure(figsize=(7,5)); plt.plot(df.nx, df.C, marker='o'); plt.gca().invert_xaxis(); plt.xlabel('grid size n_x'); plt.ylabel('coherence C'); plt.title('RG: C vs resolution'); plt.tight_layout(); plt.show()


In [ ]:

# 4) Gauged RFT gradient flow (stable parameters)
nx=96; L=2*np.pi; kappa1=1.0; kappa2=0.2; m2=0.1; dt=5e-6; steps=20000; sample_every=200
theta = 0.5*init_vortex_pair(nx,nx,L,sep_frac=0.2)
ax = np.zeros((nx,nx)); ay=np.zeros((nx,nx)); dx=L/nx
def gauge_energy(th, ax, ay):
    thx, thy = grad_periodic(th, dx)
    Dx = thx - ax; Dy = thy - ay; F = curl2d_periodic(ax, ay, dx)
    E = 0.5*kappa1*((Dx**2+Dy**2).mean()) + 0.5*kappa2*((F**2).mean())
    if m2!=0.0: E += m2*(1-np.cos(th)).mean()
    return float(E)
times=[]; Cs=[]; Es=[]
for step in range(steps+1):
    if step % sample_every==0:
        times.append(step*dt); Cs.append(coherence(theta)); Es.append(gauge_energy(theta,ax,ay))
    thx, thy = grad_periodic(theta, dx)
    Dx = thx-ax; Dy=thy-ay; F=curl2d_periodic(ax,ay,dx)
    dtheta = kappa1*div_periodic(Dx,Dy,dx) - (m2*np.sin(theta) if m2!=0 else 0.0)
    cX,cY = (np.roll(F,-1,0)-np.roll(F,1,0))/(2*dx), -(np.roll(F,-1,1)-np.roll(F,1,1))/(2*dx)
    dax = kappa1*(thx-ax) + kappa2*cX; day = kappa1*(thy-ay) + kappa2*cY
    theta = wrap_phase(theta + dt*dtheta); ax = ax + dt*dax; ay = ay + dt*day
df = pd.DataFrame({'t':times,'C':Cs,'E':Es}); df.to_csv('/mnt/data/nb_gauged_rft_timeseries.csv', index=False)
plt.figure(figsize=(7,5)); plt.plot(df.t, df.E); plt.xlabel('time'); plt.ylabel('E(t)'); plt.title('Gauged RFT gradient flow'); plt.tight_layout(); plt.show()
